# Generate Dataset from Log File

In [61]:
import os
import pandas as pd
from matplotlib import pyplot as plt

In [103]:
LOG_FILE_DIR = '/home/vinaykudari/Documents/code/HomeMotorAutomation/data/' 
FILE_NAME = 'homebridge.log'

LOG_FILE_PATH = os.path.join(LOG_FILE_DIR, FILE_NAME)
FROM_DATE = '2020-02-22'

In [148]:
def clean_log_file(log):
    log = [line.replace('\x1b[37m', '').
           replace('\x1b[39m \x1b[36m[Motor]\x1b[39m', ' : ').
           replace('\n', '').
           replace('\'','').
           replace('.', '').split(" : ")
           for line in log if 'turned' in line]
    
    return log

In [149]:
def process_log(cleaned_log, from_date):
    log_df = pd.DataFrame(log_cleaned, columns=['date', 'status'])
    log_df['datetime'] = pd.to_datetime(log_df.date, format="[%m/%d/%Y, %I:%M:%S %p]")
    log_df['date'] = log_df.datetime.apply(lambda x : x.date())
    log_df['date'] = pd.to_datetime(log_df.date, format="%Y-%m-%d") 
    
    processed_log = log_df[log_df['date'] > from_date]
    
    return processed_log

In [164]:
def transform_log(processed_log):
    log_night = processed_log.groupby(['date', 'status']).apply(max).reset_index(drop=True)
    log_morning = processed_log.groupby(['date', 'status']).apply(min).reset_index(drop=True)
    
    df_night = log_night.pivot(index='date', columns='status', values='datetime').reset_index()
    df_morning = log_morning.pivot(index='date', columns='status', values='datetime').reset_index()
    
    df = pd.concat([df_night, df_morning]).sort_values(by='date').reset_index(drop=True)
    df.columns = ['date', 'turned_off_time', 'turned_on_time']
    
    df['day_of_week'] = df['date'].dt.day_name()
    df['time_taken'] = df['turned_off_time'] - df['turned_on_time']
    
    df['turned_off_time'] = df['turned_off_time'].dt.time
    df['turned_on_time'] = df['turned_on_time'].dt.time
    
    return df[['date', 'day_of_week', 'turned_on_time', 'turned_off_time', 'time_taken']]

In [165]:
log = open(LOG_FILE_PATH).readlines()
log_cleaned = clean_log_file(log)
processed_log = process_log(log_cleaned, FROM_DATE)
df = transform_log(processed_log)

In [166]:
df

,date,day_of_week,turned_on_time,turned_off_time,time_taken
0,2020-02-23,Sunday,19:00:20,19:07:55,00:07:35
1,2020-02-23,Sunday,00:12:47,00:25:29,00:12:42
2,2020-02-24,Monday,07:05:51,07:16:31,00:10:40
3,2020-02-24,Monday,11:27:18,11:44:36,00:17:18
4,2020-02-25,Tuesday,12:37:32,12:54:24,00:16:52
...,...,...,...,...,...
177,2020-05-21,Thursday,20:14:22,20:21:26,00:07:04
178,2020-05-22,Friday,06:42:44,06:56:32,00:13:48
179,2020-05-22,Friday,13:47:06,13:54:56,00:07:50
180,2020-05-23,Saturday,20:20:01,20:36:23,00:16:22
